# Import

In [ ]:
from google.colab import drive
import pandas as pd
import matplotlib.pyplot as plt
import plotly.graph_objects as go
import numpy as np
import matplotlib.dates as mdates
import re

# per creare tutte le combinazioni di anno e mese
import itertools

# Cleansing

In [ ]:
# Read csv
df = pd.read_csv('<qui inserisci il path che puoi recuperare facendo tasto dx e copia path>')

In [ ]:
df.shape # per vedere quante righe e colonne hai, così verifichi se hai importato correttamente nella variabile df

## Drop cols

In [ ]:
def drop_cols(df):

    """
    Funzione per pulire un DataFrame rimuovendo colonne e righe indesiderate.

    Parametri:
        df (pd.DataFrame): Il DataFrame da pulire.

    Ritorna:
        pd.DataFrame: Il DataFrame pulito.
    """

    # 1. Lista manuale delle colonne da eliminare, rispettando l'ordine del csv
    manual_columns_to_drop = [
        'Financial Status', 'Paid at', 'Fulfillment Status', \
        'Fulfilled at', 'Currency', \
        'Total', 'Shipping Method', 'Lineitem sku', \
        'Lineitem requires shipping', 'Lineitem taxable', 'Lineitem fulfillment status', \
        'Shipping Name'
    ]

    # Droppa le colonne della lista manuale
    df = df.drop(columns=[col for col in manual_columns_to_drop if col in df.columns], errors='ignore')

    # 2. Parole chiave da cercare nei nomi delle colonne
    keywords = ['Billing', 'Street', 'Address', 'Company', 'Zip', 'Country', 'Phone', 'Note']

    # Filtra e droppa le colonne che contengono una qualsiasi delle parole chiave
    keyword_columns_to_drop = [col for col in df.columns if any(keyword in col for keyword in keywords)]
    df = df.drop(columns=keyword_columns_to_drop, errors='ignore')

    # 3. Elimina tutte le colonne da "Payment Method" fino alla fine
    if "Payment Method" in df.columns:
        start_col = df.columns.get_loc("Payment Method")
        df = df.drop(columns=df.columns[start_col:], errors='ignore')

    # 4. Elimina la riga che contiene un valore nella colonna "Cancelled at"
    # QUI FALLO SOLO SE DEVI TOGLIERE DALL'ANALISI GLI ORDINI CHE POI SONO STATI CANCELLATI
    if 'Cancelled at' in df.columns:
        df = df[df['Cancelled at'].isna()]
        df = df.drop(columns=['Cancelled at'], errors='ignore')

    return df

In [ ]:
df = drop_cols(df)

## DType mapping

In [ ]:
# starting point dtypes
df.info()

In [ ]:
def convert_type(df):

    """
    Funzione per eseguire conversioni di tipo e pulizia su un DataFrame.

    Parametri:
        df (pd.DataFrame): Il DataFrame da pulire e convertire.

    Ritorna:
        pd.DataFrame: Il DataFrame con i tipi di dati aggiornati e le colonne modificate.
    """

    # Conversioni a stringa
    string_columns = ['Name', 'Order Number', 'Email', 'Lineitem name', 'Shipping City']
    for col in string_columns:
        if col in df.columns:
            df[col] = df[col].astype('string')

    # Conversioni a booleano
    if 'Accepts Marketing' in df.columns:
        df['Accepts Marketing'] = df['Accepts Marketing'].astype(bool)

    # Conversioni a categoria
    category_columns = ['Discount Code', 'Shipping Province']
    for col in category_columns:
        if col in df.columns:
            df[col] = df[col].astype('category')

    # Pulizia e conversione della colonna 'Created at'
    if 'Created at' in df.columns:
        # Rimuove il fuso orario
        df['Created at'] = df['Created at'].str.replace(r' \+\d{4}', '', regex=True)
        # Converte in datetime
        df['Created at'] = pd.to_datetime(df['Created at'], errors='coerce')

    # Conversione della colonna 'Taxes' in int8
    if 'perc_taxes' in df.columns:
        df['perc_taxes'] = df['perc_taxes'].astype('int8')

    return df

In [ ]:
df = convert_type(df)

In [ ]:
# Check
df.info()

## Rename cols

In [ ]:
df.rename(columns={'Name': 'Order Number', 'Taxes': 'perc_taxes'}, inplace=True)
# inplace=True: sovrascrive il df senza dichiarare una nuova variabile df
# inplace=False: comportamento di default, necessario creare un'ulteriore variabile df

## Rename values

In [ ]:
def replace_lineitem_names(df):

    """
    Funzione per sostituire valori specifici nella colonna 'Lineitem name'.

    Parametri:
        df (pd.DataFrame): Il DataFrame da modificare.

    Ritorna:
        pd.DataFrame: Il DataFrame con i valori sostituiti.
    """

    # Dizionario delle sostituzioni
    replacements = {
        'questo': 'in questo',
        'questaltro': 'in questaltro ancora'
    }

    # Esegui le sostituzioni
    if 'Lineitem name' in df.columns:
        df['Lineitem name'] = df['Lineitem name'].replace(replacements)

    return df

In [ ]:
df = replace_lineitem_names(df)

In [ ]:
# Check per alfabetically order
sorted(df["Lineitem name"].unique())

## Feature Eng

In [ ]:
df.info()

In [ ]:
def enrich_datetime_columns(df, datetime_column='Created at'):

    """
    Funzione per arricchire un DataFrame con informazioni temporali estratte da una colonna datetime.

    Parametri:
        df (pd.DataFrame): Il DataFrame da modificare.
        datetime_column (str): Nome della colonna datetime da elaborare.

    Ritorna:
        pd.DataFrame: Il DataFrame con le nuove colonne temporali aggiunte e convertite.
    """

    if datetime_column in df.columns:
        # Estrai informazioni temporali
        df['Hour'] = df[datetime_column].dt.hour
        df['Day'] = df[datetime_column].dt.day
        df['Month'] = df[datetime_column].dt.month
        df['Year'] = df[datetime_column].dt.year
        df['Day of Week'] = df[datetime_column].dt.day_name()
        df['Date'] = df[datetime_column].dt.date

        # Converte la colonna Date in datetime64
        df['Date'] = pd.to_datetime(df['Date'])

    return df

In [ ]:
df = enrich_datetime_columns(df)

In [ ]:
df.info()

In [ ]:
# Creiamo la nuova colonna basata sui valori di 'Lineitem name'
# QUESTO è PER CREARE LA COLONNA CATEGORIA PRODOTTI

df['Category'] = df['Lineitem name'].apply(
    lambda x: 'Bundle' if '+' in x else  # Se il nome del prodotto contiene "+", è un bundle
              'Y' if 'X' in x else  # Se il nome del prodotto contiene X allora Y
              'Altro'
)

In [ ]:
df['Category'] = df['Category'].astype('category')

In [ ]:
print(df['Category'].cat.categories)  # Mostra le categorie uniche

## Add Taxes col manually

In [ ]:
# Aggiornare la colonna 'Taxes' in base alla condizione sulla colonna "Lineitem name"
df["perc_taxes"] = df["Lineitem name"].apply(lambda x: 10 if "X" in x.lower() or "Y" in x.lower() else 22)
# = inserisci 10% se il nome prodotto contiene X o Y, altrimenti il 22%

In [ ]:
# aggiorna il tipo di dato che deve essere a virgola mobile
df['perc_taxes'] = df['perc_taxes'].astype('float64')

## Add `Subtotal after taxes` col

In [ ]:
df["Subtotal after taxes"] = df["Subtotal"] - (df["Subtotal"] / 100 * df["perc_taxes"])

In [ ]:
df['Subtotal after taxes'] = df['Subtotal after taxes'].astype('float64')

## Save in parquet format to retain dtypes

In [ ]:
df.to_parquet('<nome che vuoi dare al file>.parquet')

# Analysis

In [ ]:
df = pd.read_parquet('<path di quello che hai salvato nella cella prima>')

In [ ]:
# verifica un'anteprima
df.head()

In [ ]:
df.info()

## Total orders

In [ ]:
total_orders = df["Order Number"].nunique()
total_orders

## Total unique customers

In [ ]:
unique_customers = df['Email'].nunique()
unique_customers

## Recurrent customers %

In [ ]:
rec_customers = total_orders - unique_customers
rec_customers

In [ ]:
# in percentuale rispetto al totale
round(rec_customers/unique_customers*100,1)

## Best 3 province

In [ ]:
# best 3 province
df["Shipping Province"].value_counts().head(3)

## Number of scatole di x sold

In [ ]:
# Filtra le righe che contengono "X" (case insensitive)
df_X = df[df['Lineitem name'].str.contains('X', case=False, na=False)].copy()

In [ ]:
# Funzione per estrarre il numero, altrimenti 1
def extract_quantity(text):
    match = re.search(r'\b(\d+)\b', text)
    return int(match.group(1)) if match else 1

In [ ]:
# Applica la funzione e crea la nuova colonna 'quantita'
df_X['quantita'] = df_X['Lineitem name'].apply(extract_quantity)

In [ ]:
quantita_totale = df_X['quantita'].sum()
print(quantita_totale)

In [ ]:
df_X['quantita'].value_counts()

## Unique orders df

In [ ]:
unique_orders = df.drop_duplicates(subset=['Order Number'])

## Average daily order per hour

In [ ]:
# Calcoliamo la media giornaliera degli ordini per ciascun orario
average_daily_orders_per_hour = unique_orders.groupby(['Date', 'Hour']).size().groupby('Hour').mean()

In [ ]:
# Creare un DataFrame completo con tutte le ore
complete_hours = pd.DataFrame({'Hour': range(24)})

# Verifica che 'average_daily_orders_per_hour' sia un DataFrame
# Questo step non è sempre necessario, ma utile per evitare problemi
if isinstance(average_daily_orders_per_hour, pd.Series):
    average_daily_orders_per_hour = average_daily_orders_per_hour.reset_index()
    average_daily_orders_per_hour.columns = ['Hour', 'Average Orders']

# Effettuare il merge
average_daily_orders_per_hour = pd.merge(
    complete_hours,
    average_daily_orders_per_hour,
    on='Hour',
    how='left'
)

# Riempire i valori mancanti con 0
average_daily_orders_per_hour['Average Orders'] = average_daily_orders_per_hour['Average Orders'].fillna(0)

In [ ]:
plt.figure(figsize=(10, 6))

# Istogramma
plt.bar(average_daily_orders_per_hour['Hour'], average_daily_orders_per_hour['Average Orders'])

# Linea di tendenza
plt.plot(
    average_daily_orders_per_hour['Hour'],
    average_daily_orders_per_hour['Average Orders'],
    color='orange',
    marker='o',
    label='Trendline'
)

plt.title("Average Daily Orders per Hour")

plt.xlabel("Hour of the Day")
plt.ylabel("Average Daily Orders")

plt.xticks(range(24))

plt.tight_layout()

plt.show()

## Average daily order per day of the week

In [ ]:
# Raggruppiamo per 'Date' e 'Day of Week' per calcolare gli ordini giornalieri
orders_per_day = unique_orders.groupby(['Date', 'Day of Week']).size().reset_index(name='Order Count')

In [ ]:
# Raggruppiamo per 'Day of Week' e calcoliamo la media
average_orders_per_day_of_week = orders_per_day.groupby('Day of Week')['Order Count'].mean()

In [ ]:
# Definizione dell'ordine naturale dei giorni della settimana
days_order = ["Monday", "Tuesday", "Wednesday", "Thursday", "Friday", "Saturday", "Sunday"]

# Riordinare i dati in base ai giorni della settimana
average_orders_per_day_of_week = average_orders_per_day_of_week.reindex(days_order)

plt.figure(figsize=(10, 6))

# Grafico a barre
plt.bar(
    average_orders_per_day_of_week.index,
    average_orders_per_day_of_week.values,
    label='Bar Chart'
)

# Linea di tendenza
plt.plot(
    average_orders_per_day_of_week.index,
    average_orders_per_day_of_week.values,
    color='orange',
    marker='o',
    linestyle='-',
    linewidth=2,
    label='Trendline'
)

# Personalizzazione del grafico
plt.title('Media giornaliera degli ordini per giorno della settimana', fontsize=16)

plt.xlabel('Giorno della settimana', fontsize=12)
plt.ylabel('Media ordini giornalieri', fontsize=12)

# Ottimizzazione layout
plt.xticks(rotation=45)  # Ruota i giorni per migliorare la leggibilità
plt.tight_layout()

# Mostra il grafico
plt.show()

## Number of orders per day

In [ ]:
# Raggruppa per giorno e conta il numero di ordini unici
unique_orders_daily = unique_orders.groupby(unique_orders['Date'])['Order Number'].nunique()

# Creiamo un intervallo continuo di date per assicurare che tutti i giorni siano rappresentati
date_range = pd.date_range(start=unique_orders_daily.index.min(), end=unique_orders_daily.index.max())
unique_orders_daily = unique_orders_daily.reindex(date_range, fill_value=0)
unique_orders_daily.index = pd.to_datetime(unique_orders_daily.index)  # Assicuriamo che l'indice sia datetime

# Definiamo le annotazioni, AGGIORNATE AL 9 APR
annotations = {
    "2024-08-25": "Evento 1",
    "2024-09-01": "Evento 2"
}

# Convertiamo le annotazioni in una Serie Pandas per mappare le date
annotations = {pd.to_datetime(date): note for date, note in annotations.items()}

# Creiamo il DataFrame con le informazioni
unique_orders_daily = pd.DataFrame({
    'orders': unique_orders_daily,
    'annotations': [annotations.get(date, '') for date in unique_orders_daily.index]
})

# Funzione per assegnare colori ai punti in base al tipo di evento
def assign_color(annotation):
    if "Consegna" in annotation:
        return 'green'  # 🟢 Consegne
    elif "@" in annotation:
        return 'red'  # 🔴 Market o evento
    return 'blue'  # 🔵 Default per tutti gli altri ordini

# Applica la funzione per determinare i colori dei punti
unique_orders_daily['color'] = unique_orders_daily['annotations'].apply(assign_color)

# Creazione del grafico interattivo
fig = go.Figure()

# Aggiungiamo i punti con colori dinamici
fig.add_trace(go.Scatter(
    x=unique_orders_daily.index,
    y=unique_orders_daily['orders'],
    mode='lines+markers',
    name='Ordini unici',
    marker=dict(
        color=unique_orders_daily['color'],  # Colori aggiornati
        size=10
    ),
    hovertemplate=(
        '%{x|%Y-%m-%d}<br>'
        'Ordini: %{y}<br>'
        '<b>Annotazioni:</b> %{customdata}<extra></extra>'
    ),
    customdata=unique_orders_daily['annotations']  # Passa le annotazioni come dati personalizzati
))

# Aggiungiamo le due linee verticali per disegnare la finestra temporale della promo
# Definiamo le date (assicurati che queste siano all'interno del range del grafico)
promo_inizio_date = pd.Timestamp("2025-04-07")
promo_fine_date   = pd.Timestamp("2025-04-14")

fig.add_shape(
    type="line",
    x0=promo_inizio_date,
    x1=promo_inizio_date,
    y0=0,
    y1=1,
    xref="x",
    yref="paper",  # y in riferimento all'area del grafico (da 0 a 1)
    line=dict(
        color="blue",
        width=2,
        dash="dash"
    )
)

fig.add_shape(
    type="line",
    x0=promo_fine_date,
    x1=promo_fine_date,
    y0=0,
    y1=1,
    xref="x",
    yref="paper",
    line=dict(
        color="blue",
        width=2,
        dash="dash"
    )
)

# Aggiungiamo le annotazioni per le linee verticali, posizionate sopra il grafico
fig.add_annotation(
    x=promo_inizio_date,
    y=1.02,
    xref="x",
    yref="paper",
    text="inizio promo 1",
    showarrow=False,
    font=dict(color="blue")
)

fig.add_annotation(
    x=promo_fine_date,
    y=1.02,
    xref="x",
    yref="paper",
    text="fine promo 1",
    showarrow=False,
    font=dict(color="blue")
)

# Imposta il layout del grafico
fig.update_layout(
    title='Time Series: Numero di Ordini Unici per Giorno',
    xaxis_title='Data',
    yaxis_title='Numero di Ordini',
    hovermode='x unified',  # Mostra un solo popup per asse x
    template='plotly_white',
    margin=dict(l=40, r=40, t=60, b=40)  # Margini del grafico
)

# Mostra il grafico
fig.show()

## Best product category

In [ ]:
df['Category'].value_counts().head(20)

## Numero di ordini per nome maglietta

In [ ]:
# Filtra il DataFrame per considerare solo le T-shirts
tshirt_data = df[df['Category'] == 'T-shirts'].copy()

# Estrai la taglia (tra il trattino e lo slash) e rimuovi spazi bianchi
tshirt_data['Taglia'] = tshirt_data['Lineitem name'].str.extract(r'- ([^/]+)')

# Estrai il colore (dopo lo slash) e rimuovi spazi bianchi
tshirt_data['Colore'] = tshirt_data['Lineitem name'].str.extract(r'/ (.+)$')

# Estrai il nome della maglietta (prima del secondo trattino)
tshirt_data['Nome Maglietta'] = tshirt_data['Lineitem name'].str.extract(r'^T-shirt\s+"(.*?)"')

In [ ]:
# Assegna "Se stai leggendo" alle righe con NaN nella colonna 'Nome Maglietta'
tshirt_data.loc[tshirt_data['Nome Maglietta'].isna(), 'Nome Maglietta'] = "aaa"

In [ ]:
# Check
tshirt_data[tshirt_data['Nome Maglietta'].isna()][["Lineitem name", "Nome Maglietta"]]
# OK

,Lineitem name,Nome Maglietta


In [ ]:
tshirt_data['Nome Maglietta'].value_counts()

In [ ]:
# replace "X" con "Y"
tshirt_data['Nome Maglietta'] = tshirt_data['Nome Maglietta'].replace({'X': 'Y'})

In [ ]:
tshirt_data['Nome Maglietta'].value_counts()

## Numero di ordini per mug

In [ ]:
# Filtra il DataFrame per considerare solo le Mug
mug_data = df[df['Category'] == 'Mugs'].copy()

In [ ]:
mug_data['Lineitem name'].value_counts()

## `Subtotal after taxes` per year and month

In [ ]:
df.head()

In [ ]:
# Raggruppare per Year e Month sommando Subtotal after taxes
grouped_df = df.groupby(['Year', 'Month'])['Subtotal after taxes'].sum().round(1).reset_index()

In [ ]:
grouped_df